In [1]:
import torch
import cv2
from torchvision import transforms
import numpy as np
import glob
import os
import tqdm
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
weigths = torch.load('yolov7-w6-infantpose.pt')
model = weigths['model']
model = model.half().to(device)
_ = model.eval()
torch.set_grad_enabled(False)
def process_keypoints(video_file, model):
    video = cv2.VideoCapture(video_file)
    num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm.tqdm(total=num_frames, desc="inf")
    keypoints=[]
    while video.isOpened():
        ret, frame = video.read()
        #print(frame.shape)
        if (frame is None):
            break

        pbar.update(1)
        
        box = letterbox(frame, 1024, stride=64, auto=True)
        frame=box[0]
        frame_ = frame.copy()
        frame = transforms.ToTensor()(frame)
        frame = torch.tensor(np.array([frame.numpy()]))
        frame = frame.to(device)
        frame = frame.half()

        output, _ = model(frame)

        output = non_max_suppression_kpt(output, 0.25, 0.45, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)[0]
        try:    
            total_conf=output[:,8::3].sum(1,keepdim=True)
            _, j = total_conf.max(0, keepdim=True)
            output=output[int(j),:]
            output=output.unsqueeze(0)
        except:
            try:
                output=last_output
                keypoints.append(output)
                continue
            except:
                output=np.zeros((1,57))
                keypoints.append(output)
                continue
        
        with torch.no_grad():
            output = output_to_keypoint(output)
        last_output=output.copy()
        keypoints.append(output)
    
    video.release()
    keypoints_array=np.stack(keypoints)
    return keypoints_array


video_path = './data/video/infant.mp4'
output_path = './data/results_npy/'

vidname = os.path.basename(video_path)
vname = vidname.split('.')[0]
keypoints=process_keypoints(video_path, model)
print(keypoints.shape)
np.save(output_path+vname+'.npy',keypoints)

c:\Users\Yw\anaconda3\envs\test1\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
inf:   0%|          | 0/1000 [00:00<?, ?it/s]c:\Users\Yw\anaconda3\envs\test1\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
inf: 100%|██████████| 1000/1000 [01:23<00:00, 12.04it/s]

(1000, 1, 57)
